### Arrow Experiments
https://arrow.apache.org/docs/python/index.html

Arrow Memory Pools

In [1]:
import pyarrow as pa

data = b'abcdefghijklmnopqrstuvwxyz'
buf = pa.py_buffer(data)
buf
buf.size

26

In [3]:
memoryview(buf)

In [4]:
buf.to_pybytes()

b'abcdefghijklmnopqrstuvwxyz'

In [5]:
pa.total_allocated_bytes()

0

In [7]:
buf = pa.allocate_buffer(1024, resizable=True)
pa.total_allocated_bytes()

1024

In [8]:
buf.resize(2048)
pa.total_allocated_bytes()

2048

Input Streams

In [9]:
buf = memoryview(b"some data")
stream = pa.input_stream(buf)
stream.read(4)

b'some'

In [10]:
import gzip
with gzip.open('example.gz', 'wb') as f:
    f.write(b'some data\n' * 3)
stream = pa.input_stream('example.gz')
stream.read()

b'some data\nsome data\nsome data\n'

Output Streams

In [11]:
with pa.output_stream('example1.dat') as stream:
    stream.write(b'some data')

f = open('example1.dat', 'rb')

f.read()

b'some data'

CSV Files

In [12]:
from pyarrow import csv
fn = 'tips.csv.gz'
table = csv.read_csv(fn)
table
pyarrow.Table
len(table)
df = table.to_pandas()
df.head()

FileNotFoundError: [Errno 2] Failed to open local file 'tips.csv.gz'. Detail: [errno 2] No such file or directory